In [2]:
import sys
print(sys.version)

3.11.4 (main, Aug 10 2024, 18:44:41) [Clang 14.0.0 (clang-1400.0.29.202)]


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import numpy as np

In [5]:
!pwd

/Users/dtolqin/Documents/ADVANCED MACHINE LEARNING APPLICATIONS/adv_mla_2024/adv_mla_lab_1/notebooks


In [6]:
df = pd.read_csv('../data/raw/insurance.csv')

In [7]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,18,female,33.820,0,no,southeast,1630.66170
1,19,female,23.480,1,no,southeast,1836.80430
2,46,male,30.570,2,no,southeast,6632.35130
3,54,male,32.050,1,yes,southeast,31922.42950
4,21,male,21.345,4,no,northeast,1638.37255


In [8]:
df.shape

(50000, 7)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       50000 non-null  int64  
 1   sex       50000 non-null  object 
 2   bmi       50000 non-null  float64
 3   children  50000 non-null  int64  
 4   smoker    50000 non-null  object 
 5   region    50000 non-null  object 
 6   charges   50000 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 2.7+ MB


In [10]:
df.describe()

,age,bmi,children,charges
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,39.463120,30.713734,1.113760,13343.216363
std,14.117142,6.092727,1.212835,12131.222744
min,18.000000,17.291000,0.000000,1137.535900
25%,27.000000,26.600000,0.000000,4694.431800
50%,40.000000,30.300000,1.000000,9399.232775
75%,51.000000,34.570000,2.000000,17340.746925
max,64.000000,47.740000,5.000000,52589.810300


In [11]:
df_cleaned = df.copy()

In [12]:
target = df_cleaned.pop('charges')

In [13]:
num_cols = list(df_cleaned.select_dtypes('number').columns)
cat_cols = list(set(df_cleaned.columns) - set(num_cols))

In [14]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [15]:
ohe = OneHotEncoder(sparse_output=False, drop=('first'))

In [16]:
features = ohe.fit_transform(df_cleaned[cat_cols])

In [17]:
features = pd.DataFrame(features, columns=ohe.get_feature_names_out())

In [18]:
scaler = StandardScaler()

In [19]:
features

,region_northwest,region_southeast,region_southwest,sex_male,smoker_yes
0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,1.0
4,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...
49995,0.0,1.0,0.0,0.0,0.0
49996,1.0,0.0,0.0,1.0,0.0
49997,0.0,0.0,0.0,1.0,1.0
49998,0.0,0.0,1.0,1.0,0.0


In [20]:
features[num_cols] = scaler.fit_transform(df_cleaned[num_cols])

In [21]:
features

,region_northwest,region_southeast,region_southwest,sex_male,smoker_yes,age,bmi,children
0,0.0,1.0,0.0,0.0,0.0,-1.520374,0.509837,-0.918321
1,0.0,1.0,0.0,0.0,0.0,-1.449537,-1.187286,-0.093798
2,0.0,1.0,0.0,1.0,0.0,0.463050,-0.023591,0.730725
3,0.0,1.0,0.0,1.0,1.0,1.029743,0.219324,-0.093798
4,0.0,0.0,0.0,1.0,0.0,-1.307864,-1.537707,2.379771
...,...,...,...,...,...,...,...,...
49995,0.0,1.0,0.0,0.0,0.0,0.746396,-0.300974,0.730725
49996,1.0,0.0,0.0,1.0,0.0,1.313089,0.885699,0.730725
49997,0.0,0.0,0.0,1.0,1.0,-0.812008,-1.069931,-0.093798
49998,0.0,0.0,1.0,1.0,0.0,1.242253,-0.346931,0.730725


In [22]:
from joblib import dump

In [23]:
!ls ../

Makefile         docs             pyproject.toml   setup.cfg
README.md        models           references
adv_mla_lab_1    notebooks        reports
data             poetry.lock      requirements.txt


In [24]:
dump(ohe, '../models/ohe.joblib')
dump(scaler, '../models/scaler.joblib')

['../models/scaler.joblib']

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
x_data, x_test, y_data, y_test = train_test_split(features,
                                                  target,
                                                  test_size=0.2,
                                                  random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data,
                                                  test_size=0.2,
                                                  random_state=8)


In [27]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(32000, 8)
(8000, 8)
(10000, 8)


In [28]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(32000,)
(8000,)
(10000,)


In [29]:
x_train.to_csv('../data/processed/X_train.csv', index=False)
x_val.to_csv('../data/processed/X_val.csv', index=False)
x_test.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_val.to_csv('../data/processed/y_val.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

In [30]:
pred_value = y_train.mean()

In [31]:
y_base = np.full((len(y_train),1), pred_value)

In [32]:
y_base

array([[13364.5344162],
       [13364.5344162],
       [13364.5344162],
       ...,
       [13364.5344162],
       [13364.5344162],
       [13364.5344162]])

In [33]:
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.metrics import mean_absolute_error as mae

In [34]:
print(rmse(y_base, y_train))
print(mae(y_base, y_train))

12165.591800038708
9156.906457632496


### Push Cha